In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
import pandas as pd

In [3]:
# Download the data - 106 GB !!!
# !kaggle competitions download -c siim-isic-melanoma-classification

In [4]:
path = Config.data_path()/'melclass'
path.mkdir(parents=True, exist_ok=True)
path
bs = 64

In [5]:
!ls /home/jupyter/.fastai/data/melclass/jpeg

test  train


In [6]:
fnames = get_image_files(path/'jpeg/test')
fnames[:3]

[PosixPath('/home/jupyter/.fastai/data/melclass/jpeg/test/ISIC_9128800.jpg'),
 PosixPath('/home/jupyter/.fastai/data/melclass/jpeg/test/ISIC_6731469.jpg'),
 PosixPath('/home/jupyter/.fastai/data/melclass/jpeg/test/ISIC_9754437.jpg')]

In [9]:
import random
random.seed(42)
traincsv = pd.read_csv(path/'train.csv')
target0 = traincsv.query("target == 0")
target1 = traincsv.query("target == 1")
subset_t0 = target0.iloc[random.sample(range(len(target0)), len(target1)*2),]
subset = pd.concat([subset_t0, target1])
subset.head()
subset.to_csv(path/'subset.csv', index=False)

In [10]:
subset.target.value_counts()

0    1168
1     584
Name: target, dtype: int64

In [11]:
data = ImageDataBunch.from_csv(
    path, 
    'jpeg/train', 
    csv_labels='subset.csv',
    suffix='.jpg', 
    valid_pct=0.5,
    fn_col=0,
    label_col=7,
    size=112, # 224
    bs=64,
    ds_tfms=get_transforms()
).normalize(imagenet_stats) # need to use imagenet stats because this is the one the model was trained with

In [12]:
# data.show_batch(rows=5, figsize=(7,6))

In [13]:
print(data.classes)
len(data.classes),data.c

[0, 1]


(2, 2)

In [14]:
learn = cnn_learner(data, models.resnet34, metrics=[error_rate, dice, AUROC()])

In [ ]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,error_rate,dice,auroc,time
0,1.023035,0.740207,0.271690,0.728311,0.778528,02:51


In [ ]:
learn.save('melclass-stage-1')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()
len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, max_lr=slice(1e-5,1e-3))

Note, can also use freeze_to to unfreeze last x layers (e.g. freeze_to(-2) to unfreeze the last two layers)) - perhaps only relevant for NLP

In [ ]:
learn.save('melclass-stage-2')